In [1]:
import os
import numpy as np
import dill
import pickle
from bottleneck import push
from fancyimpute import IterativeImputer
from miss_utils import *

In [2]:
data_path = 'extracts/'
with open(os.path.join(data_path, '3d_tensor_set_a.pkl'), 'rb') as f:
    origional_a = pickle.load(f)
    
with open(os.path.join(data_path, '3d_tensor_set_b.pkl'), 'rb') as f:
    origional_b = pickle.load(f)    

with open(os.path.join(data_path, '3d_tensor_set_c.pkl'), 'rb') as f:
    origional_c = pickle.load(f)
    
origional = np.vstack([origional_a,origional_b,origional_c]) 
mask=~np.isnan(origional)*1

In [3]:

mins = []
maxs = []
flatten= origional.reshape(origional.shape[0]*origional.shape[1], origional.shape[2])

for i in range(origional.shape[2]):

        mins.append(np.nanmin(flatten[:,i]))

        maxs.append(np.nanmax(flatten[:,i]))

normalized =normalize(origional,mins,maxs)

Patient level introduce

In [4]:
seed = 1234
new_tensor, o,miss_indices=array_rsme(normalized,mask, 0.1, seed)
LO =push(new_tensor, axis=1)
LOCV= np.where(np.isnan(LO), 0, LO)
zero= np.where(np.isnan(new_tensor), 0, new_tensor)
means, stds = [],[]
flatten= new_tensor.reshape(origional.shape[0]*origional.shape[1], 35)

for i in range(35):
    means.append(np.nanmean(flatten[:,i]))
    stds.append(np.nanstd(flatten[:,i]))
mean_imputed =mean_fill(new_tensor, means)
mask_miss=~np.isnan(new_tensor)*1
imputed=prepare_fills(new_tensor, means)
IMM= create_individualized_missingness_mask(mask_miss)

In [5]:
# with open('extracts/X_0.1_'+str(seed)+'.pkl', 'wb') as outfile:
#     dill.dump(new_tensor, outfile, pickle.HIGHEST_PROTOCOL)  
# with open('extracts/new_miss_oo_0.1_'+str(seed)+'.pkl', 'wb') as outfile:
#     dill.dump(zero, outfile, pickle.HIGHEST_PROTOCOL) 
# with open('extracts/new_miss_0.1_back_forward_'+str(seed)+'.pkl', 'wb') as outfile:
#     dill.dump(imputed, outfile, pickle.HIGHEST_PROTOCOL) 
# with open('extracts/mask_miss_0.1_'+str(seed)+'.pkl', 'wb') as outfile:
#     dill.dump(mask_miss, outfile, pickle.HIGHEST_PROTOCOL)  
# with open('extracts/IMM_0.1_'+str(seed)+'.pkl', 'wb') as outfile:
#     dill.dump(IMM, outfile, pickle.HIGHEST_PROTOCOL)  

In [6]:
mice_impute = IterativeImputer()
flatten= new_tensor.reshape(new_tensor.shape[0]*new_tensor.shape[1], new_tensor.shape[2])
MICE = mice_impute.fit_transform(flatten)
MICE=MICE.reshape(new_tensor.shape[0], new_tensor.shape[1], new_tensor.shape[2])
MICE =MICE.copy()

In [18]:
experiment_name = "IMM_population_1234_01"
saved_path = 'data/'
observed_only_real = np.load(saved_path + experiment_name + '.npz')["observed_only_real"]
observed_only_rec = np.load(saved_path + experiment_name + '.npz')["observed_only_rec"]
IMM_real = np.load(saved_path + experiment_name + '.npz')["IMM_real"]
IMM_rec = np.load(saved_path + experiment_name + '.npz')["IMM_rec"]
imputed_ours =(mask_miss * (zero)+ ((1-mask_miss) *IMM_rec))
GP = np.load("GP_0.1.npy")
GP_ =(mask_miss * (zero)+ ((1-mask_miss) *GP))

with open(os.path.join('SAITS_01_miss_ratio.pkl'), 'rb') as f:
           SAITS = pickle.load(f)    
with open(os.path.join('BRITS_01_miss_ratio.pkl'), 'rb') as f:
            BRITS = pickle.load(f)  
        
experiment_name = "final_loss_contrastive_added_to_top_miss_01_pop"
saved_path = 'data/'
observed_only_real2 = np.load(saved_path + experiment_name + '.npz')["observed_only_real"]
observed_only_rec2 = np.load(saved_path + experiment_name + '.npz')["observed_only_rec"]
IMM_real2 = np.load(saved_path + experiment_name + '.npz')["IMM_real"]
IMM_rec2 = np.load(saved_path + experiment_name + '.npz')["IMM_rec"]       

In [19]:
imputed_ours2 =(mask_miss * (zero)+ ((1-mask_miss) *observed_only_rec))

In [20]:
imputed_ours =(mask_miss * (zero)+ ((1-mask_miss) *IMM_rec))
imputed_ours2 =(mask_miss * (observed_only_real)+ ((1-mask_miss) *observed_only_rec))
imputed_ours =(mask_miss * (observed_only_real)+ ((1-mask_miss) *IMM_rec))

imputed_ours_oo =(mask_miss * (observed_only_real)+ ((1-mask_miss) *observed_only_rec2))
imputed_our_imm =(mask_miss * (observed_only_real)+ ((1-mask_miss) *IMM_rec2))

In [21]:
MICE =MICE.copy()

In [22]:
flat= np.zeros((12000,48,35)).reshape(-1)
flat[miss_indices] = 1
indicating_mask=flat.reshape(12000,48,35)


In [23]:
print("MAE")
print("----------------------")
get_patient_level_MAE_array(LOCV, normalized, indicating_mask, "LOCF") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(imputed_ours, normalized, indicating_mask, "IGNITE") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(imputed_ours2, normalized, indicating_mask, "IGNITE top") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_patient_level_MAE_array(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

MAE
----------------------
LOCF 0.05165800462934476 0.027114409211606072
mean 0.057670157153456546 0.020238543335535392
mice 0.04616320078828389 0.018753096791934355
BRITS 0.0024177451679315734 0.0035662082574153572
SAITS 0.0025955762507195324 0.003548599447711563
IGNITE 0.13910672629481463 0.04361542216578956
IGNITE top 0.04989488488943224 0.01775066692252501
IGNITE new OO 0.05450862826601374 0.018317351187256768
IGNITE new IMM 0.14860325561304014 0.043433446060300626


In [24]:
print("RMSE")
print("----------------------")
get_patient_level_RMSE_array(LOCV, normalized, indicating_mask, "LOCF") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(imputed_ours, normalized, indicating_mask, "IGNITE OLD ") # cl
get_patient_level_RMSE_array(imputed_ours2, normalized, indicating_mask, "IGNITE OLD top") # cl
get_patient_level_RMSE_array(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_patient_level_RMSE_array(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

RMSE
----------------------
LOCF 0.114883608165942 0.06284515936606051
mean 0.09354327473848718 0.037884700165760905
mice 0.08538658971605063 0.041727681138876566
BRITS 0.0097380219805777 0.01350374072975449
SAITS 0.010214848651670341 0.013343438752790296
IGNITE OLD  0.20587205256933677 0.05328527492066753
IGNITE OLD top 0.07630776711985905 0.03265895350440511
IGNITE new OO 0.08354316449394361 0.033867877067840196
IGNITE new IMM 0.21693774693254397 0.053564702090987265


In [25]:
print("MSE")
print("----------------------")
get_patient_level_MSE_array(LOCV, normalized, indicating_mask, "LOCF") 
get_patient_level_MSE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(imputed_ours, normalized, indicating_mask, "IGNITE OLD ") # cl
get_patient_level_MSE_array(imputed_ours2, normalized, indicating_mask, "IGNITE OLD top") # cl
get_patient_level_MSE_array(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_patient_level_MSE_array(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

MSE
----------------------
LOCF 0.017147757480971235 0.018922169036915223
mean 0.0101855947554497 0.008945209646617276
mice 0.009032069076564918 0.010381120623681718
BRITS 0.0002771800857906448 0.001239675258724628
SAITS 0.00028239049072599726 0.0012648536726619574
IGNITE OLD  0.04522262255248288 0.02312208951350993
IGNITE OLD top 0.006889482566821537 0.00665812334366644
IGNITE new OO 0.008126493430744456 0.00716633381079762
IGNITE new IMM 0.049931163354264695 0.02425188413960078


Calculate Population level sum

In [26]:
get_population_RMSE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_RMSE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(imputed_ours, normalized, indicating_mask, "IGNITE") # cl
get_population_RMSE(imputed_ours2, normalized, indicating_mask, "IGNITE top") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_population_RMSE(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

LOCF 0.12938261893970704
mean 0.10222940397185304
mice 0.09660854988723191
BRITS 0.016449460954054876
SAITS 0.016612336892042848
IGNITE 0.2116896978597842
IGNITE top 0.08384295218008475
IGNITE new OO 0.09094525887899133
IGNITE new IMM 0.22270213336997438


In [27]:
get_population_MSE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MSE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(imputed_ours, normalized, indicating_mask, "IGNITE lower") # cl
get_population_MSE(imputed_ours2, normalized, indicating_mask, "IGNITE top") # cl
get_population_MSE(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_population_MSE(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

LOCF 0.016739862083697436
mean 0.010450851036440322
mice 0.009333211911313779
BRITS 0.000270584765678976
SAITS 0.0002759697370147278
IGNITE lower 0.044812528179966726
IGNITE top 0.007029640630271977
IGNITE new OO 0.008271040112566751
IGNITE new IMM 0.04959624020753786


In [28]:
get_population_MAE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MAE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(imputed_ours, normalized, indicating_mask, "IGNITE") # cl
get_population_MAE(imputed_ours2, normalized, indicating_mask, "IGNITE top") # cl
get_population_MAE(imputed_ours_oo, normalized, indicating_mask, "IGNITE new OO") # cl
get_population_MAE(imputed_our_imm, normalized, indicating_mask, "IGNITE new IMM") # cl

LOCF 0.05076275454231429
mean 0.05789591910258009
mice 0.046517276436382315
BRITS 0.0023792980604556402
SAITS 0.0025681972851686594
IGNITE 0.13807257666174003
IGNITE top 0.05008819766921089
IGNITE new OO 0.054669810965794804
IGNITE new IMM 0.14775936398853798


get weighted population sum

In [21]:
get_population_MAE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MAE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)


LOCF 0.050762754540736796
mean 0.057895919100818996
mice 0.046517276434805806
BRITS 0.0023792980603818065


NameError: name 'get_populatiaon_MAE_weighted' is not defined

In [20]:
get_population_MAE_weighted(imputed_ours2, normalized, indicating_mask, "IGNITE lower")# calculate mean absolute error on the ground truth (artificially-missing values)


IGNITE lower 0.050088197667687163


In [22]:
get_population_MAE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

SAITS 0.0025681972850893986
IGNITE 0.13807257665749245


In [ ]:
get_population_RMSE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_RMSE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

In [ ]:
get_population_MSE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MSE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

In [ ]:
# def get_population_RMSE_weighted(imputation,observed, indicating_mask, name):
#     array_RMSE = []
#     array_weight =[]
#     observed = np.where(np.isnan(observed), 0, observed)

#     for count in range(imputation.shape[0]):
#          selected = indicating_mask[count]
#          w= (selected.sum())/(indicating_mask.sum())
#          array_weight.append(w)
#          if len(selected)> 0:
#            mse=np.sum(np.square(imputation[count] - observed[count]) * selected) / (np.sum(selected) + 1e-9)
#            rms=np.sqrt(mse)
#          else:
#              rms = 0.0
#          array_RMSE.append(rms*w)
#     print(name, np.sum(array_RMSE))

In [80]:
imputed_ours_oo[10,:20,1]

array([0.267462015, 0.265432656, 0.280000001, 0.296666652, 0.189999998,
       0.256423354, 0.310000002, 0.293333322, 0.340000004, 0.353333324,
       0.403333336, 0.353333324, 0.409999996, 0.343333334, 0.356666654,
       0.336666673, 0.340000004, 0.250196338, 0.250149906, 0.323333323])

In [81]:
imputed_ours[10,:20,1]

array([0.235224307, 0.265464514, 0.280000001, 0.296666652, 0.189999998,
       0.264504522, 0.310000002, 0.293333322, 0.340000004, 0.353333324,
       0.403333336, 0.353333324, 0.409999996, 0.343333334, 0.356666654,
       0.336666673, 0.340000004, 0.256831676, 0.256266147, 0.323333323])

In [82]:
#introduced 
IMM[10,:20,1]

array([0.833333333, 0.833333333, 1.         , 1.         , 1.         ,
       0.833333333, 1.         , 1.         , 1.         , 1.         ,
       1.         , 1.         , 1.         , 1.         , 1.         ,
       1.         , 1.         , 0.833333333, 0.833333333, 1.         ])

In [83]:
normalized[10,:20,1]

array([        nan, 0.323333333, 0.28       , 0.296666667, 0.19       ,
       0.3        , 0.31       , 0.293333333, 0.34       , 0.353333333,
       0.403333333, 0.353333333, 0.41       , 0.343333333, 0.356666667,
       0.336666667, 0.34       , 0.32       , 0.343333333, 0.323333333])